<a href="https://colab.research.google.com/github/LuisIrigoyen/trabajo_final_RL/blob/main/proyecto_final_APR_V06_4_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models', 'checkpoints', 'dqn_05_weights.h5f.data-00000-of-00001', 'dqn_05_weights.h5f.index', 'dqn_06_weights.h5f.index', 'dqn_06_weights.h5f.data-00000-of-00001', 'dqn_07_weights.h5f.index', 'dqn_07_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.index', '.ipynb_checkpoints', 'dqn_09_weights.h5f.index', 'dqn_09_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.index', 'dqn_06_3_weights.h5f.data-00000-of-00001', 'd

## 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.05
  %pip install Keras==2.2.4
  %pip install tensorflow==2.1.0
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-653ug49k
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-653ug49k
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Dropout
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):

        if batch.ndim == 4:
            batch = np.expand_dims(batch, axis=-1)
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 1. EJFM Implementación de la red neuronal

In [7]:
def build_model(height, width, channels, actions):

    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), activation='relu', input_shape=(4,84,84,1)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [8]:
model = build_model(84, 84, 1, nb_actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 20, 20, 32)     2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 9, 9, 64)       32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 7, 7, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 512)               6423040   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 3

## 2. EJFM Implementación de la solución DQN

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),
        attr='eps',
        value_max=1.0,
        value_min=0.1,
        value_test=0.005,
        nb_steps=200000)
    memory = SequentialMemory(limit=100000, window_length=4)
    dqn = DQNAgent(model=model,
                   memory=memory,
                   policy=policy,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   nb_actions=nb_actions,
                   batch_size=32,
                   target_model_update=1000,
                   processor=AtariProcessor(),
                   nb_steps_warmup=10000
                  )
    return dqn

In [16]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 10,000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v06_4_weights_{step}.h5f',
    interval=10000,
    verbose=1
)

In [15]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [17]:
from keras.optimizers import Adam

dqn = build_agent(model, nb_actions)
dqn.compile(Adam(learning_rate=0.00025, epsilon=1e-4))


In [ ]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=300000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_06_4_weights.h5f')


⏱ Tiempo de inicio: 20:19:39
Training for 300000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    544/300000: episode: 1, duration: 3.807s, episode steps: 544, steps per second: 143, episode reward:  6.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.511 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1695/300000: episode: 2, duration: 8.447s, episode steps: 1151, steps per second: 136, episode reward: 13.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 2.384 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2197/300000: episode: 3, duration: 3.273s, episode steps: 502, steps per second: 153, episode reward:  8.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.420 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2746/300000: episode: 4, duration: 3.545s, episode steps: 549, steps per second: 155, episode reward:  8.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.362 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3436/300000: episode: 5, duration: 5.846s, episode steps: 690, steps per second: 118, episode reward: 10.000

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  10332/300000: episode: 16, duration: 98.078s, episode steps: 586, steps per second:   6, episode reward:  6.000, mean reward:  0.010 [ 0.000,  1.000], mean action: 2.473 [0.000, 5.000],  loss: 0.006575, mean_q: 0.047341, mean_eps: 0.954253
  10721/300000: episode: 17, duration: 112.418s, episode steps: 389, steps per second:   3, episode reward:  9.000, mean reward:  0.023 [ 0.000,  1.000], mean action: 2.517 [0.000, 5.000],  loss: 0.006254, mean_q: 0.045360, mean_eps: 0.952633
  11487/300000: episode: 18, duration: 222.191s, episode steps: 766, steps per second:   3, episode reward:  9.000, mean reward:  0.012 [ 0.000,  1.000], mean action: 2.470 [0.000, 5.000],  loss: 0.007191, mean_q: 0.076031, mean_eps: 0.950034
  12027/300000: episode: 19, duration: 155.822s, episode steps: 540, steps per second:   3, episode reward:  4.000, mean reward:  0.007 [ 0.000,  1.000], mean action: 2.504 [0.000, 5.000],  loss: 0.007931, mean_q: 0.093204, mean_eps: 0.947096
  12837/300000: episode: 20, 

In [ ]:
# Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_06_4_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 7.000, steps: 510
Episode 2: reward: 10.000, steps: 650
Episode 3: reward: 16.000, steps: 890
Episode 4: reward: 11.000, steps: 583
Episode 5: reward: 13.000, steps: 619
Episode 6: reward: 14.000, steps: 813
Episode 7: reward: 10.000, steps: 476
Episode 8: reward: 27.000, steps: 1077
Episode 9: reward: 12.000, steps: 524
Episode 10: reward: 34.000, steps: 1592


In [ ]:
import gym
from gym.wrappers import Monitor
import os

video_dir = './video'
os.makedirs(video_dir, exist_ok=True)

env = gym.make("SpaceInvaders-v0")
env = Monitor(env, video_dir, force=True)


In [ ]:
obs = env.reset()
done = False
while not done:
    action = dqn.forward(obs)  # acción del modelo
    obs, reward, done, info = env.step(action)
env.close()


UnboundLocalError: cannot access local variable 'processed_batch' where it is not associated with a value

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    video = open(path,'rb').read()
    encoded = b64encode(video).decode('ascii')
    return HTML(f"""
      <video width=512 controls>
            <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
      </video>""")

# Mostrar el video grabado
import glob
video_files = glob.glob(os.path.join(video_dir, "*.mp4"))
show_video(video_files[0])


In [ ]:
from google.colab import drive
import shutil
import glob  # <-- ¡Falta este import!
import os

# 1. Montar Google Drive
#drive.mount('/content/drive')

# 2. Crear directorio en Drive si no existe
os.makedirs('/content/gdrive/MyDrive/Colab_Outputs', exist_ok=True)

# 3. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('dqn_06_4_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )
    print(f"¡Archivo {archivo} copiado a Drive!")

# 4. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('checkpoints/dqn_v06_4_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )

print("✅ ¡Todos los archivos copiados!")

¡Archivo dqn_06_weights.h5f.data-00000-of-00001 copiado a Drive!
¡Archivo dqn_06_weights.h5f.index copiado a Drive!
✅ ¡Todos los archivos copiados!
